In [504]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
%matplotlib inline

In [505]:
#Sigmoid Function to return the sigmoid values of all elements in a matrix

def sigmoid(Z):
    g=1/(1+np.exp(-Z))
    return(g)

In [506]:
#Function to randomly initialize the parameters(weights) of the NN

def randinit(input_size,hidden_size,n_hidden,n_labels):
    
    theta=[] #Setting up the empty list that will store the matrices of parameters(weights) for every layer of the NN
    e=0.6 #range of initialization [-e,+e]

    for i in range(n_hidden+1):

            if i==0:
                theta.append(((np.random.rand(hidden_size,input_size+1))*2*e)-e)#Parameters to move from input layer to first hidden layer

            elif i==n_hidden:
                theta.append(((np.random.rand(n_labels,hidden_size+1))*2*e)-e)#Parameters to move between intermediate hidden layers 

            else:
                theta.append(((np.random.rand(hidden_size,hidden_size+1))*2*e)-e)#Parameters to move from last hidden layer to output layer

    return(theta)

In [507]:
#Function to unroll parameters from list of matrices to a single row arra

def unroll(M):

    a=[]
    arr=np.array([])
    
    for i in range(len(M)):
        M[i]=M[i].flatten()
        arr=np.concatenate((arr,M[i]))
    a.append(arr.tolist())
    a=np.array(a)

    return(a)

In [508]:
def roll(params,input_size,hidden_size,n_hidden,n_labels,use0):
    
    theta=[] 
    theta_grad=[] #Setting up an empty list to have arrays of derivatives of parameters for the layers of the NN
    count=0 #Setting up a counter to track indexing for params when rolling
    params_sum=0
    
    for i in range(n_hidden+1):
        
        if i==0:
            theta.append(np.reshape(params[0:(hidden_size*(input_size+1))],(hidden_size,input_size+1)))
            count=count+(hidden_size*(input_size+1))
            
        elif i==n_hidden:
            theta.append(np.reshape(params[count:count+(n_labels*(hidden_size+1))],(n_labels,hidden_size+1)))
            
        else:
            theta.append(np.reshape(params[count:count+(hidden_size*(hidden_size+1))],(hidden_size,hidden_size+1)))
            count=count+(hidden_size*(hidden_size+1))
                              
        if use0==1: #This is only used inside the objective/cost function
            
            theta_grad.append(np.zeros(np.shape(theta[i]))) #Setting up the gradient matrix
        
            r=np.copy(theta[-1])
            params_sum=params_sum+np.sum(np.power(r[:,1:],2)) #Adding all the squares of the parameters for regularization
        
    return(theta,theta_grad,params_sum)

In [509]:
#Computing the cost/objective function and gradients for the neural network (NN)

def computeCostandGrad(params,input_size,hidden_size,n_hidden,n_labels,X,y,lmbda):
    
    params=np.array(params) #Converting the parameters to a matrix
    params=params.transpose() #Converting the row matrix to a column matrix
    
    m=np.shape(X)[0] #The number of training examples
    J=0 #The Objective/Cost Function value 
    
    #To reobtain the parameters/weights for the layers of the NN
    (theta,theta_grad,params_sum)=roll(params,input_size,hidden_size,n_hidden,n_labels,1) 
    
    I=np.identity(n_labels) #Setting up vectors for the labels for checking in output layer of the NN
    
    a=[] #Setting up an empty list to have arrays of activation values of the various layers of the NN
    
    #Finding the matrices of activation values
    
    for i in range(n_hidden+1):
        
        if i==0:
            a.append(sigmoid((np.column_stack((np.ones(np.shape(X)[0]),X))).dot(theta[i].transpose()))) 
        else:
            a.append(sigmoid((np.column_stack((np.ones(np.shape(a[i-1])[0]),a[i-1]))).dot(theta[i].transpose())))
    
    #Setting up the y matrix (output matrix of training data)
    
    yi=np.zeros((m,n_labels))
    
    for i in range(m):
        yi[i,:]=I[y[i],:]
    
    #for i in range(m):
        #if y[i]==0:
            #yi[i,:]=I[n_labels-1,:]
        #else:
            #yi[i,:]=I[y[i]-1,:]
        
    #Finding the value of the objective/cost function
    
    J=(-1/m)*(np.sum((np.multiply(yi,np.log(a[-1])))+(np.multiply((1-yi),np.log((1-a[-1]))))))
    
    #Regularization of cost function
    
    J=J+((lmbda/(2*m))*params_sum)
    
    #Backpropagation to evaluate gradients
    
    delta=[] #Setting up empty list to store error matrices for the various layers of the NN
    
    for i in range(n_hidden+1):
        
        #Calculating the error values
        
        if i==0:
            delta.append(a[-1]-yi)
        else:
            k=np.multiply(a[-1-i],(1-a[-1-i]))
            delta.append(np.multiply(delta[i-1].dot(theta[-1-(i-1)]),np.column_stack((np.ones(np.shape(k)[0]),k))))
            k=delta[-1]
            delta[-1]=k[:,1:]
        
        #Computing the gradients (backpropagation)
        
        if i==n_hidden:
            theta_grad[-1-i]=(1/m)*((delta[-1].transpose()).dot(np.column_stack((np.ones(np.shape(X)[0]),X))))
        else:
            theta_grad[-1-i]=(1/m)*((delta[-1].transpose()).dot(np.column_stack((np.ones(np.shape(a[i])[0]),a[i]))))
            
        #Regularization of gradient term
        
        k=np.copy(theta_grad[-1-i])
        t=np.copy(theta[-1-i])
        k[:,1:]=k[:,1:]+((lmbda/(m))*t[:,1:])
        theta_grad[-1-i]=k
            
    #Unrolling gradients

    grad=unroll(theta_grad)
    
    return(J,grad)

    

In [510]:
#Making sure that the gradients are being computed properly
#Important to check that back-propagation is running properly as it is prone to errors

def gradientCheck(lmbda):
    
    #Creating a simplified mini-neural network with single hidden layer for the checking process
    
    input_size=2
    hidden_size=4
    n_labels=2
    n_hidden=1
    m=5 #Number of training examples for the NN
    e=0.5 #Limits of the random input data
    
    #Setting up the learning parameters
    
    theta1=((np.random.rand(hidden_size,input_size+1))*2*e)-e
    theta2=((np.random.rand(n_labels,hidden_size+1))*2*e)-e
    theta_nn=np.concatenate((theta1.flatten(),theta2.flatten())) #Converting into a one dimensional array
    
    #Initializing random X and Y
    
    X=np.random.rand(m,input_size)
    y=np.random.randint(0,n_labels,size=(m,1))
    
    (J_nn,grad_nn)=computeCostandGrad(theta_nn,input_size,hidden_size,n_hidden,n_labels,X,y,lmbda)
    
    epsilon=0.00001 #Constant for central difference
    
    grad_nn=grad_nn.transpose()
    grad_cd=np.zeros((np.shape(grad_nn)))
    theta_cd=np.copy(theta_nn)
    
    for i in range(np.shape(grad_nn)[0]):
               
            
        #Finding the forward and backward terms for central differencing
        
        theta_cd[i]=theta_cd[i]+epsilon
        (J_plus,not_used)=computeCostandGrad(theta_cd,input_size,hidden_size,n_hidden,n_labels,X,y,lmbda)
        theta_cd[i]=theta_cd[i]-(2*epsilon)
        (J_minus,not_used)=computeCostandGrad(theta_cd,input_size,hidden_size,n_hidden,n_labels,X,y,lmbda)
      
        grad_cd[i]=(J_plus-J_minus)/(2*epsilon) #Central Difference formula
        
        theta_cd=np.copy(theta_nn)
    
    print('''Gradients obtained for a trial mini neural network through the neural network and cetral difference
    schemes have been compared as shown by below''')
    a=np.concatenate((grad_nn,grad_cd),axis=1)
    print(a)
    print('\n')
    avr=(np.sum(grad_nn-grad_cd))/(np.shape(grad_nn)[0])
    print('The average difference is ',avr)
    print('\n')
    print('''An error of order lesser than 10^-9 is sufficient enough to hold confidence on the implementation
    of the backpropagation code''')    
    


In [511]:
gradientCheck(5) #The regularization parameter is passed into the gradientCheck function

Gradients obtained for a trial mini neural network through the neural network and cetral difference
    schemes have been compared as shown by below
[[-0.00170624 -0.00170624]
 [-0.43181396 -0.43181396]
 [-0.35662139 -0.35662139]
 [-0.0333351  -0.0333351 ]
 [ 0.16163533  0.16163533]
 [ 0.38530481  0.38530481]
 [-0.02790602 -0.02790602]
 [-0.31309449 -0.31309449]
 [ 0.30911627  0.30911627]
 [ 0.01668689  0.01668689]
 [-0.47553238 -0.47553238]
 [-0.26904231 -0.26904231]
 [ 0.18053257  0.18053257]
 [-0.26994559 -0.26994559]
 [ 0.02933429  0.02933429]
 [ 0.0834642   0.0834642 ]
 [-0.24701927 -0.24701927]
 [-0.28900878 -0.28900878]
 [-0.28378036 -0.28378036]
 [ 0.30066277  0.30066277]
 [ 0.31117     0.31117   ]
 [-0.54898338 -0.54898338]]


The average difference is  6.41529522056e-13


An error of order lesser than 10^-9 is sufficient enough to hold confidence on the implementation
    of the backpropagation code


In [512]:
data=pd.read_excel('DigitData.xlsx')

#Separating X and Y data

cols=data.shape[1] #.shape is a tuple hence you call with []. 0 for row size and 1 for column size
X=data.iloc[:,0:cols-1] #iloc can be used to access the data in various ways
y=data.iloc[:,cols-1:cols]
X=np.array(X)
y=np.array(y) 

In [513]:
#To generate random parameter values for the layers of NN

theta=randinit(input_size,hidden_size,n_hidden,n_labels) 
theta_unroll=unroll(theta) #Unrolling the parameters to be used in the NN cost and gradient function

In [535]:
#Initializations

#-----------------------------------------------USER PROVIDED---------------------------------------------------------------#
hidden_size=25  #Number of nodes in the hidden layers
n_hidden=1  #Number of hidden layers
n_labels=10  #Number of labels in dataset for classification
lmbda=5 #Regularization Parameter
#---------------------------------------------------------------------------------------------------------------------------#

input_size=np.shape(X)[1] #Number of nodes in the input layer

#To generate random parameter values for the layers of NN

theta=randinit(input_size,hidden_size,n_hidden,n_labels) 
theta_unroll=unroll(theta) #Unrolling the parameters to be used in the NN cost and gradient function
#To find Value of cost function before optimization
(J_init,grad_init)=computeCostandGrad(theta_unroll,input_size,hidden_size,n_hidden,n_labels,X,y,lmbda)

#Neural Network Learning
print(type(theta_unroll))
fmin=minimize(fun=computeCostandGrad,x0=theta_unroll,args=(input_size,hidden_size,n_hidden,n_labels,X,y,lmbda),method='BFGS',jac=True)
theta_final_unrolled=fmin.x
J_final=fmin.fun #Value of cost function after optimization


print('The optimization process is complete\n')
print('The initial value of the cost/objective function =',J_init)
print('The final value of the cost/objective function =',J_final)


<class 'numpy.ndarray'>


ValueError: shapes (10285,10285) and (1,10285) not aligned: 10285 (dim 1) != 1 (dim 0)

In [515]:
#Obtaining the optimized parameters/weights of the NN layers
(theta_opt,not_used,not_used)=roll(theta_final_unrolled,input_size,hidden_size,n_hidden,n_labels,0)

In [516]:
#To predict the accuracy of the NN by checking with given input data

m=np.shape(X)[0] #The number of training examples

for i in range(len(theta_opt)):   
    k=np.copy(theta_opt[i])
    k=k.transpose()
    
    if i==0:     
        h=sigmoid(np.column_stack((np.ones(np.shape(X)[0]),X)).dot(k))
    else:
        h=sigmoid(np.column_stack((np.ones(np.shape(h)[0]),h)).dot(k))

y_predict=np.matrix(np.argmax(h,axis=1))
y_predict=y_predict.transpose()
accuracy=(1-(((np.count_nonzero(y_predict-y)))/len(y_predict)))*100
print('The accuracy of the neural network is',accuracy,'%')


The accuracy of the neural network is 96.39999999999999 %


The weights/parameters for the layers of the neural network are stored in theta_opt, which is a list of arrays. The first array corresponds to the first layer and so on.

In [517]:
print(y_predict)

[[0]
 [0]
 [0]
 ..., 
 [9]
 [9]
 [9]]
